In [1]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [2]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class3.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [3]:
def valueCheck(x):
    value = x.split(",")
    length = len(value)
    return length
def has26Values(x):
    if x == 26:
        return True
    else:
        return False
for grade in grades: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                incorrect_id = value[0]
                print("Invalid line of data:", (incorrect_id) + " does not contain exactly 26 values:")
                print(grade)    
values_count = list(map(valueCheck,grades))
values_check = list(map(has26Values,values_count))
print("Total valid lines of data:", values_check.count(True))
print("Total invalid lines of data:", values_check.count(False))


Total valid lines of data: 100
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html 
https://stackoverflow.com/questions/53837841/collect-rows-based-on-unique-id-pandas-dataframe

--------------------
count value of unique answer
count if value != True

In [26]:
from os import rename


answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
answers_checked = []
for grade in grades:
        value = grade.split(",")
        values = []
        if len(value) > 26:
                continue
        elif len(value) < 25:
                continue
        else:
                answer_table = pd.DataFrame(value)
                answer_table = answer_table.iloc[1: , :]
                answer_table = answer_table.reset_index(drop=True)
                cVal = answer_table.iloc[24][0]
                cValSplit = cVal.split("\n")
                c = cValSplit[0]
                answer_table = answer_table.replace([cVal],c)
                answer_table['checkAnswer'] = np.where(answer_table[0] == answer_key_table[0], 'True', answer_key_table[0])
                answers_checked.append(answer_table)

                        

a = pd.concat(answers_checked)
a.columns = ['Student Answer', 'Check Answer']
new_df = a.groupby('Check Answer')

new_df



Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'